<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Load data to Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>


<p style = 'font-size:16px;font-family:Arial'>Welcome to this introductory guide. This guide will walk you through steps  are needed to load data in Teradata from ipython SQL kernel.  A more extensive notebook with more commands can be found online in the Teradata Vantage Modules for Jupyter <a href = 'https://github.com/Teradata/jupyterextensions/blob/master/notebooks/sql/DataLoad.ipynb'>Here.</a>  

<hr style="height:2px;border:none;">
<p style = 'font-size:16px;font-family:Arial'>For help on available "magic" commands in the Teradata SQL plugin, run the following cell:</p>

In [ ]:
%help

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press Enter, then use down arrow to go to next cell.</p>

In [ ]:
%connect local, hidewarnings=true

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'> <b>2. Load data from file present locally </b></p>

<p style = 'font-size:16px;font-family:Arial'>Apart from the standard Teradata utilities for data load i.e TPT; Teradata SQL plugin for Jupyter has a magic command to load data into the database.</p>

In [ ]:
%help dataload

<p style = 'font-size:16px;font-family:Arial'>The data we will be loading for this example is in CSV format. The following is a sample of the header followed by the first 5 rows:</p>

<p style = 'font-size:12px;font-family:Courier'>
InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country<br>
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom<br>
536365,71053,WHITE METAL LANTERN,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom<br>
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,2.75,17850.0,United Kingdom<br>
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom<br>
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom<br>
</p>

<p style = 'font-size:16px;font-family:Arial'>We will need to create a table that matches the columns of the data in our file.</p

In [ ]:
CREATE MULTISET TABLE DEMO_USER.Retail_Data 
       (InvoiceNo VARCHAR(10) CHARACTER SET UNICODE NOT CASESPECIFIC,
        StockCode VARCHAR(10) CHARACTER SET UNICODE NOT CASESPECIFIC,
        Description VARCHAR(40) CHARACTER SET UNICODE NOT CASESPECIFIC,
        Quantity BIGINT,
        InvoiceDate TIMESTAMP(6),
        UnitPrice FLOAT,
        CustomerID FLOAT,
        Country VARCHAR(25) CHARACTER SET UNICODE NOT CASESPECIFIC)
  PRIMARY INDEX ( InvoiceNo );

In [ ]:
%dataload DATABASE = demo_user, TABLE = Retail_Data, skiprows = 1,
FILEPATH = Getting_Started/Data_Loading/Retail_Data_sample.csv

<p style = 'font-size:16px;font-family:Arial'>Let's check the count and sample records loaded. </p>


In [ ]:
SELECT COUNT(*) FROM Retail_Data;

In [ ]:
SELECT TOP 5 * FROM Retail_Data;

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'> <b>3. Load data from file present in cloud </b></p>

<p style = 'font-size:16px;font-family:Arial'>We can access and load files present in cloud via Vantage's NOS (Native Object Store). For this we need the access id and key for authentication and the location off the file. Please refer <a href ='https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-VantageTM-Native-Object-Store-Getting-Started-Guide-17.20/Reading-Data/Examples-For-Business-Analysts/Sampling-External-Data-In-Place-Using-READ_NOS'>NOS user guide</a>  for more information.<br>
  First of all we need to create an Authorization object to access the file. </p> 

<p style = 'font-size:16px;font-family:Arial'>The command template to create an authorization object is below. If you are loading data from your cloud storage, you would need to substitute for the values in the {  } brackets. </p>

<p style = 'font-size:12px;font-family:Courier'> CREATE AUTHORIZATION {name} AS DEFINER TRUSTED<br>
USER '{access_key}'<br>
PASSWORD '{password}';<br>
</p>
<p style = 'font-size:16px;font-family:Arial'>We have created authorization object gs_tables_db.PUBLIC_AUTH for our pulic buckets and we will use them in the below queries.</p>

<hr style="height:1px;border:none;">

<p style = 'font-size:18px;font-family:Arial'><b>3.1 Sampling External Data</b><br>

<p style = 'font-size:16px;font-family:Arial'>
Our file Retail_Data_sample.csv is located at /gs/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/<br>
<p style = 'font-size:16px;font-family:Arial'>We can preview the contents of an external object store by using the command below. The location string is prefixed indicating where the data is located.
<ul>
    <li> "/gs" - to access files in google cloud storage, used below</li>
    <li> "/s3" - to access AWS buckets (Used in the UseCases/Deep_History_via_Object_Store)</li>
    <li> "/az" - to access Azure blob storage</li>
</ul>


In [ ]:
SELECT TOP 2 * FROM (
LOCATION = '/gs/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/Retail_Data_sample.csv'
AUTHORIZATION=gs_tables_db.PUBLIC_AUTH    
) AS D;

<p style = 'font-size:16px;font-family:Arial'> Please keep in mind:
<ul style = 'font-size:16px;font-family:Arial'><li>TOP does not return the same results each time it is run.</li>
<li>TOP may return records with only a subset of attributes because not all records in different objects contain all of the same attributes.</li>
    </ul>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.2 Using Foreign Table</b>

In [ ]:
CREATE FOREIGN TABLE Retail_Data_FT,
EXTERNAL SECURITY gs_tables_db.PUBLIC_AUTH
USING (LOCATION('/gs/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/Retail_Data_sample.csv'));

In [ ]:
SELECT TOP 5 * FROM Retail_Data_FT;

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.3 Loading External Data in permanent table</b><br>
<p style = 'font-size:16px;font-family:Arial'>If we want to copy data from cloud to permanent table in database we can use the below command

In [ ]:
CREATE MULTISET TABLE DEMO_USER.Retail_Data_Perm 
       (InvoiceNo VARCHAR(10) CHARACTER SET UNICODE NOT CASESPECIFIC,
        StockCode VARCHAR(10) CHARACTER SET UNICODE NOT CASESPECIFIC,
        Description VARCHAR(40) CHARACTER SET UNICODE NOT CASESPECIFIC,
        Quantity BIGINT,
        InvoiceDate TIMESTAMP(6),
        UnitPrice FLOAT,
        CustomerID FLOAT,
        Country VARCHAR(25) CHARACTER SET UNICODE NOT CASESPECIFIC)
  PRIMARY INDEX ( InvoiceNo );

In [ ]:
INSERT DEMO_USER.Retail_Data_Perm 
SELECT
    InvoiceNo,
    StockCode,
    Description,
    Quantity,
    InvoiceDate,
    UnitPrice,
    CustomerID,
    Country
FROM (
    LOCATION = '/gs/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/Retail_Data_sample.csv'
    AUTHORIZATION=gs_tables_db.PUBLIC_AUTH 
) AS d;

In [ ]:
SELECT TOP 5 * FROM Retail_Data_Perm;

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>3.4 Parquet Data on AWS</b><br>
<p style = 'font-size:16px;font-family:Arial'>As mentioned previously, several of the demonstration notebooks access data on AWS. One of them is:</p>

[Deep History via Object Store](../../UseCases/Deep_History_via_Object_Store/Deep_History_via_Object_Store_SQL.ipynb)

<p style = 'font-size:16px;font-family:Arial'>The data it accesses is in parquet format. Storage of data in the cloud is frequently broken up into multiple files to allow parallel reading of the data for performance and to allow some partitioning of the data using the directory hierarchy (in this case by year). The data is public so it can be accessed without an Authorization object. The data for that demo is in 28 files which we can view with the following command. </p>

In [ ]:
SELECT TOP 5 location(char(255)), ObjectLength 
FROM read_nos (
USING 
 LOCATION  ('/s3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/') 
 AUTHORIZATION (gs_tables_db.PUBLIC_AUTH )   
 RETURNTYPE ('NOSREAD_KEYS')
) AS d 
ORDER BY 1;

<p style = 'font-size:16px;font-family:Arial'>The definition of the data can be read using the RETURNTYPE 'NOSREAD_PARQUET_SCHEMA'.   In this case we are only going to reference 1 file out of the set for performance.  We will specify FULLSCAN 'TRUE' to read through all rows to determine the maximum column length for varchar columns (if any ... at least for 1 file).  </p>

In [ ]:
SELECT TOP 5 * FROM READ_NOS (
      USING
      LOCATION  ('/s3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2010/2010/object_33_0_1.parquet')
      AUTHORIZATION (gs_tables_db.PUBLIC_AUTH )
      RETURNTYPE ('NOSREAD_PARQUET_SCHEMA')
      FULLSCAN ('TRUE')
      )
AS d;

<p style = 'font-size:16px;font-family:Arial'>Again, to see a sample of the data, we will only reference one file.  If you remove "2010/2010/object_33_0_1.parquet" from the location below, it will run a bit longer.</p>

In [ ]:
SELECT TOP 5 * FROM READ_NOS (
      USING
      LOCATION  ('/s3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/') --2010/2010/object_33_0_1.parquet')
      AUTHORIZATION (gs_tables_db.PUBLIC_AUTH )
      RETURNTYPE ('NOSREAD_RECORD')
      )
AS d;

<p style = 'font-size:16px;font-family:Arial'>The creating of a foreign table or loading into a local table from Parquet on the AWS cloud is similar to the process described above.  For the full example, see the UseCase:</p>

[Deep History via Object Store](../../UseCases/Deep_History_via_Object_Store/Deep_History_via_Object_Store_SQL.ipynb)

 <p style = 'font-size:16px;font-family:Arial'> To load other file formats like json or more details about parquet, please refer to: <a href ='https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-VantageTM-Native-Object-Store-Getting-Started-Guide-17.20/Reading-Data/Examples-For-Business-Analysts/Sampling-External-Data-In-Place-Using-READ_NOS'>NOS user guide</a> </p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>4. Cleanup</b>

In [ ]:
DROP TABLE Retail_Data;

In [ ]:
DROP TABLE Retail_Data_FT;

In [ ]:
DROP TABLE Retail_Data_Perm;

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>